In [1]:
import pickle
from collections import defaultdict
import numpy as np
from operator import itemgetter
import pandas as pd

In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')
stopwords = nlp.Defaults.stop_words

In [3]:
import os
#os.chdir("structural_optimization")
#os.chdir("smo")
#os.chdir("anja3")
#os.chdir("driver_glance_behaviour")
os.chdir("cov")
lvl = 0

work_dict = pickle.load(open("work_dict_lvl{}.p".format(lvl), "rb"))
#work_dict = pickle.load(open("work_dict_test_lvl1.p", "rb"))

In [4]:
#work_dict = pickle.load(open("work_dict_lvl1.p", "rb"))
#work_dict = pickle.load(open("work_dict_lvl2.p", "rb"))
#work_dict = pickle.load(open("crashworthiness/work_dict_lvl3.p", "rb"))
len(work_dict)

10939

In [5]:
from gensim.models import Word2Vec, KeyedVectors
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
def get_reference_dois(work):
    result = []
    if "reference" in work:
        for ref in work["reference"]:
            if "DOI" in ref:
                result.append(ref["DOI"])
        #if "DOI" in work:
            #result.append(work["DOI"])
    return result

In [7]:
issn_counts = defaultdict(int)
issn2journalTitle = {}

for work in work_dict.values():
    if "ISSN" in work:
        for issn in work["ISSN"]:
            issn_counts[issn] += 1
            if not issn in issn2journalTitle:
                if "container-title" in work:
                    issn2journalTitle[issn] = ";".join(work["container-title"])

In [8]:

#pd.DataFrame(index=issn_counts.keys(), data=issn_counts)
#pd.DataFrame(index=issn_counts.keys(), data={count: issn for issn, count in issn_counts.items()})
df = pd.DataFrame(index=issn_counts.keys(), columns=["Titel", "Publikationen"])
for issn, count in sorted(issn_counts.items(), key=itemgetter(1), reverse=True):
    df.at[issn, "Titel"] = issn2journalTitle[issn]
    df.at[issn, "Publikationen"] = count

df.to_excel("journal_list.xlsx")

In [9]:
#jt = set()
#f = open("journal_list.csv", "w", encoding="utf-8")
#f.write("Count\tJournalTitle\tISSN\n")
#for issn, count in sorted(issn_counts.items(), key=itemgetter(1), reverse=True):
#    if not (count, issn2journalTitle[issn]) in jt:
#        print(count, issn, issn2journalTitle[issn])
#        jt.add((count, issn2journalTitle[issn]))
#        f.write("{}\t{}\t{}\n".format(count, issn2journalTitle[issn], issn))
#f.close()

In [10]:
references_list = []
ref_counts = defaultdict(int)
for work in work_dict.values():
    #print(work)
    work_refs = get_reference_dois(work)
    #print(work_refs)
    #break
    for doi in work_refs:
        ref_counts[doi] += 1
    work_refs.append(work["DOI"])
    if len(work_refs) > 2:
        references_list.append(work_refs)
len(references_list)

5550

In [11]:
ref_counts_by_year = defaultdict(list)
for doi, count in ref_counts.items():
    #work = work_dict[doi]
    try:        
        y = work_dict[doi]["issued"]["date-parts"][0][0]
        if y:
            ref_counts_by_year[y].append(count)
    except KeyError:
        continue

In [12]:
avg_ref_count = defaultdict(int)
median_ref_count = defaultdict(int)
var_ref_count = defaultdict(int)
n_ref_count = defaultdict(int)
lambda_mle_ref_count = defaultdict(int)
percentile_ref_count = defaultdict(int)
for year, counts in ref_counts_by_year.items():
    if len(counts) > 20:
        avg_ref_count[year] = np.average(counts)
        median_ref_count[year] = np.median(counts)
        var_ref_count[year] = np.var(counts)
        n_ref_count[year] = len(counts)
        lambda_mle_ref_count[year] = len(counts) / sum(counts)
        percentile_ref_count[year] = - np.log(0.1) / lambda_mle_ref_count[year]
        
#np.average(ref_counts_by_year[2017])

sorted(avg_ref_count.items(), reverse=True)
sorted(median_ref_count.items(), reverse=True)
sorted(var_ref_count.items(), reverse=True)

lambda_mle_ref_count_default = np.median([l for y, l in lambda_mle_ref_count.items() if y < 2000])

x = sorted(zip(avg_ref_count.keys(), avg_ref_count.values(), median_ref_count.values(), var_ref_count.values(), n_ref_count.values()), reverse=True)
#pprint(x)
x = sorted(zip(avg_ref_count.keys(), avg_ref_count.values(), n_ref_count.values(), percentile_ref_count.values(), lambda_mle_ref_count.values()), reverse=True)
#print(x)
x, lambda_mle_ref_count_default


#lambda_mle_ref_count_default

C:\Users\fbockni\AppData\Local\Continuum\anaconda3\envs\new_pytorch_100\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\fbockni\AppData\Local\Continuum\anaconda3\envs\new_pytorch_100\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


([(2020, 3.6271186440677967, 236, 8.351749320351285, 0.2757009345794392),
  (2019, 1.1666666666666667, 132, 2.6863492751597198, 0.8571428571428571)],
 nan)

In [13]:
ref_count_normalized = defaultdict(float)

for doi, count in ref_counts.items():
    try:
        year = work_dict[doi]["issued"]["date-parts"][0][0]
        if year in lambda_mle_ref_count:
            if year >= 2000:
                l = lambda_mle_ref_count[year]
            else:
                l = lambda_mle_ref_count_default
        else:
            l = lambda_mle_ref_count_default
        if l < lambda_mle_ref_count_default:
            l = lambda_mle_ref_count_default
        ref_count_normalized[doi] = l*count
    except KeyError:
        continue

len(ref_counts), len(ref_count_normalized)

(130035, 378)

In [14]:
#model = Word2Vec(iter=1, size=300, window=1000, min_count=5, workers=4, sg=1)  # an empty model, no training yet
#model.build_vocab(references_list)
#model.train(references_list)

load_model = False

fname = "vectors.rv"
#fname = "testRVnew.rv"


#load_model = True

#fname = "vectors_db.rv"

if not load_model or not fname in os.listdir():
    model = Word2Vec(references_list, size=300, window=1000, min_count=3, workers=16, sg=1, iter=20)
    
    reference_vectors = model.wv
    reference_vectors.save(fname)
    del model
else:
    reference_vectors = KeyedVectors.load(fname)

2020-03-17 17:03:28,202 : INFO : collecting all words and their counts
2020-03-17 17:03:28,202 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-03-17 17:03:28,251 : INFO : collected 134546 word types from a corpus of 145148 raw words and 5550 sentences
2020-03-17 17:03:28,251 : INFO : Loading a fresh vocabulary
2020-03-17 17:03:28,297 : INFO : effective_min_count=3 retains 1374 unique words (1% of original 134546, drops 133172)
2020-03-17 17:03:28,297 : INFO : effective_min_count=3 leaves 7171 word corpus (4% of original 145148, drops 137977)
2020-03-17 17:03:28,307 : INFO : deleting the raw counts dictionary of 134546 items
2020-03-17 17:03:28,307 : INFO : sample=0.001 downsamples 32 most-common words
2020-03-17 17:03:28,307 : INFO : downsampling leaves estimated 6600 word corpus (92.1% of prior 7171)
2020-03-17 17:03:28,317 : INFO : estimated required memory for 1374 words and 300 dimensions: 3984600 bytes
2020-03-17 17:03:28,317 : INFO : resetting laye

2020-03-17 17:03:29,176 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-03-17 17:03:29,176 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-03-17 17:03:29,181 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-03-17 17:03:29,181 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-03-17 17:03:29,181 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-03-17 17:03:29,181 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-03-17 17:03:29,181 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-03-17 17:03:29,241 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-03-17 17:03:29,241 : INFO : EPOCH - 5 : training on 145148 raw words (6603 effective words) took 0.1s, 64020 effective words/s
2020-03-17 17:03:29,267 : INFO : worker thread finished; awaiting finish of 15 more threads
2020-03-17 17:03:29,272 : INFO : worker thread f

2020-03-17 17:03:29,691 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-03-17 17:03:29,691 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-03-17 17:03:29,691 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-03-17 17:03:29,691 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-03-17 17:03:29,746 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-03-17 17:03:29,746 : INFO : EPOCH - 10 : training on 145148 raw words (6617 effective words) took 0.1s, 74893 effective words/s
2020-03-17 17:03:29,756 : INFO : worker thread finished; awaiting finish of 15 more threads
2020-03-17 17:03:29,766 : INFO : worker thread finished; awaiting finish of 14 more threads
2020-03-17 17:03:29,766 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-03-17 17:03:29,771 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-03-17 17:03:29,776 : INFO : worker thre

2020-03-17 17:03:30,191 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-03-17 17:03:30,241 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-03-17 17:03:30,241 : INFO : EPOCH - 15 : training on 145148 raw words (6586 effective words) took 0.1s, 72037 effective words/s
2020-03-17 17:03:30,256 : INFO : worker thread finished; awaiting finish of 15 more threads
2020-03-17 17:03:30,266 : INFO : worker thread finished; awaiting finish of 14 more threads
2020-03-17 17:03:30,271 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-03-17 17:03:30,276 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-03-17 17:03:30,281 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-03-17 17:03:30,291 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-03-17 17:03:30,291 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-03-17 17:03:30,291 : INFO : worker th

2020-03-17 17:03:30,946 : INFO : training on a 2902960 raw words (131931 effective words) took 2.4s, 55892 effective words/s
2020-03-17 17:03:30,946 : INFO : saving Word2VecKeyedVectors object under vectors.rv, separately None
2020-03-17 17:03:30,946 : INFO : not storing attribute vectors_norm
2020-03-17 17:03:30,981 : INFO : saved vectors.rv


In [15]:
len(reference_vectors.vocab)

1374

In [16]:
#reference_vectors = model.wv
#del model

In [17]:
#fname = "vectors.rv"
#reference_vectors.save(fname)

In [18]:
import numpy as np

In [19]:
X_dict = {}

for doi in reference_vectors.vocab.keys():
    newX = {}
    newX["lvl"] = 0
    newX["vec"] = reference_vectors[doi]
    X_dict[doi] = newX
print("Size of X_dict:", len(X_dict))
if 0:
    for lvl in range(1,2):
        print ("Level:", lvl)
        dois = set(work_dict.keys() - X_dict.keys())
        print("No. DOIs:", len(dois), "/", len(work_dict))
        for doi in dois:
            work = work_dict[doi]
        #for doi, work in work_dict.items():
            if not doi in X_dict:
                ref_dois = get_reference_dois(work)
                ref_dois = set(ref_dois) & X_dict.keys()            
                if len(ref_dois) >= 3:
                    vec = np.zeros(300)
                    for ref_doi in ref_dois:
                        #print(work_dict[ref_doi])
                        vec = vec + X_dict[ref_doi]["vec"]
                    vec = vec / len(ref_dois)
                    newX = {}
                    newX["lvl"] = lvl
                    newX["vec"] = vec
                    X_dict[doi] = newX
                pass
        print("Size of X_dict:", len(X_dict))

    
    work_dict0 = pickle.load(open("work_dict_lvl0.p", "rb"))
    len(X_dict), len(work_dict0), len(work_dict), len(X_dict.keys() & work_dict.keys()), len(X_dict.keys() & work_dict0.keys()), len(reference_vectors.vocab.keys() & work_dict0.keys())

Size of X_dict: 1374


In [20]:
from sklearn.manifold import TSNE

In [21]:
len(reference_vectors.vocab.keys())

1374

In [22]:
dois_to_plot = list(reference_vectors.vocab.keys() & work_dict.keys())
#dois_to_plot = list(reference_vectors.vocab.keys())
print(len(dois_to_plot), "possible")

def doi_plot_filter(doi):
    if not "title" in work_dict[doi]:
        return False
    if len(work_dict[doi]["title"]) == 0:
        return False
    if not work_dict[doi]["issued"]["date-parts"][0][0]:
        return False
    if work_dict[doi]["issued"]["date-parts"][0][0] >= 2014:
        return True
    elif ref_counts[doi] >= 20:
        return True
    elif work_dict[doi]["issued"]["date-parts"][0][0] >= 2008 and ref_counts[doi] >= 10:
        return True
    elif work_dict[doi]["issued"]["date-parts"][0][0] >= 2012 and ref_counts[doi] >= 5:
        return True
    else:
        return False

test_dois = [
    "10.1065/lca2008.02.376",
    "10.1016/j.cirp.2010.05.004",
    #"10.1007/s11367-014-0788-0",
    "10.1007/978-3-642-01421-5",
    "10.1108/09564230310474138",
    "10.1080/17517575.2012.683812",
    "10.1002/bse.414",
    "10.1111/j.1468-2370.2007.00202.x"
]

test_dois = [
    "10.1371/journal.pone.0031092",
    "10.1093/nar/gks017",
    "10.1016/j.chembiol.2012.01.015",
    "10.1111/j.1462-2920.2012.02719.x",
    "10.1021/ja3072397",
    "10.1371/journal.ppat.1002760",
    "10.1111/1462-2920.12024",
    "10.1371/journal.ppat.1004744",
    "10.1186/1475-2859-13-88",
    "10.1016/j.mimet.2011.11.015",
    "10.1099/jmm.0.05069-0"
]

test_dois = [
    "10.1007/s001580050176", # A 99 line topology optimization code written in Matlab
    "10.1038/451652a" # Building better batteries
]

test_dois = [
    "10.1038/nature14539", # Deep learning
    "10.1038/nature16961",
    "10.1038/nature21056"
]

test_dois = [
    "10.1016/j.tcs.2014.11.028",
    "10.1007/978-3-662-05094-1",
    "10.1038/nature14422",
    "10.1126/science.1165893",
    "10.1109/4235.996017",
    "10.1007/978-3-642-17339-4"
]


test_dois = [
    "10.1016/j.addma.2016.06.010", # Topology optimization of 3D self-supporting structures for additive manufacturing
    "10.1007/s00158-015-1261-9" #Topology and shape optimization methods using evolutionary algorithms: a review
]

#test_dois = [
#    "10.1016/j.actamat.2016.07.019" # Additive manufacturing of metals
#]

#test_dois = [
#    "10.1115/1.4004465", # Multidisciplinary Design Optimization for Complex Engineered Systems: Report From a National Science Foundation Workshop
#    "10.1007/s00158-012-0763-y", # Extensions to the design structure matrix for the description of multidisciplinary design, analysis, and optimization processes
#    "10.1016/j.paerosci.2011.05.001"
#    
#]

test_dois = [
    "10.1016/j.cad.2015.04.001",
]

test_dois = [
    "10.1561/1500000011",
    "10.1145/1014052.1014073",
]

test_dois = [
    "10.1016/j.trf.2014.06.016"   # Effects of adaptive cruise control and highly automated driving on workload and situation awareness: A review of the empirical evidence
]

test_dois = [
    "10.1126/sciadv.1602614", #Data-driven discovery of partial differential equations
    #"10.1146/annurev-fluid-010816-060042",
    "10.1063/1.4927765",
    "10.1017/jfm.2016.615"
]

for doi in test_dois:
    if doi in work_dict:
        print(work_dict[doi]["title"])

def doi_plot_filter2(doi, min_sim=0.35):
    
    
    #min_sim = 0.35
    
    for test_doi in test_dois:
        if reference_vectors.similarity(doi, test_doi) > min_sim:
            return True
    return False


#dois_to_plot = [doi for doi, count in sorted(ref_count_normalized.items(), key=itemgetter(1), reverse=True)][:3000]
        
#dois_to_plot = [doi for doi in dois_to_plot if doi_plot_filter(doi)]
#dois_to_plot = [doi for doi in dois_to_plot if doi_plot_filter2(doi, 0.4)]

dois_to_plot = [doi for doi in dois_to_plot if "title" in work_dict[doi]]
dois_to_plot = [doi for doi in dois_to_plot if len(work_dict[doi]["title"]) > 0]
dois_to_plot = [doi for doi in dois_to_plot if work_dict[doi]["issued"]["date-parts"][0][0]]
dois_to_plot = [doi for doi in dois_to_plot if work_dict[doi]["issued"]["date-parts"][0][0] >= 2014]
#dois_to_plot = [doi for doi in dois_to_plot if ref_counts[doi] >= 4]

#filter_issn = "2071-1050" # Sustainability
#filter_issn = "1615-1488" # SMO
#filter_issn = "0268-3768"
#dois_to_plot = [doi for doi in dois_to_plot if "ISSN" in work_dict[doi]]
#dois_to_plot = [doi for doi in dois_to_plot if filter_issn in work_dict[doi]["ISSN"]]

#test_dois = ["10.1016/j.actamat.2016.02.014"] # Laser powder-bed fusion additive manufacturing: Physics of complex melt flow and formation mechanisms of pores, spatter, and denudation zones

#dois_to_plot = [doi for doi in dois_to_plot if doi_plot_filter2(doi, 0.8)]

print(len(dois_to_plot), "selected")

dois_to_plot = [doi for doi in dois_to_plot if doi in ref_count_normalized]
#dois_to_plot = [doi for doi in dois_to_plot if ref_count_normalized[doi] >= 1.5]

#dois_to_plot = [doi for doi, count in sorted([doi, ref_count_normalized[doi] for doi in dois_to_plot], key=itemgetter(1), reverse=True)][:3000]

a = [(doi, ref_count_normalized[doi]) for doi in dois_to_plot]
b = sorted(a, key=itemgetter(1), reverse=True)[:2500]
dois_to_plot = [doi for doi, count in b]

print(len(dois_to_plot), "selected")


122 possible
120 selected
120 selected


In [23]:
X = reference_vectors[dois_to_plot]

tsne = TSNE(n_components=2, metric="cosine")
X_embedded = tsne.fit_transform(X)
#X_embedded = TSNE(n_components=2).fit_transform(X)


In [24]:
if 0:
    dois_to_plot = list(X_dict.keys() & work_dict0.keys())
    print(len(dois_to_plot))
    X = np.zeros((len(dois_to_plot), 300))
    for i, doi in enumerate(dois_to_plot):
        X[i, :] = X_dict[doi]["vec"]

    X_embedded = TSNE(n_components=2).fit_transform(X)

In [25]:
C_embedded = TSNE(n_components=3, metric="cosine").fit_transform(X)
#C_embedded = TSNE(n_components=3).fit_transform(X)


In [26]:
#X_embedded *= 3.5

In [27]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#colors_r = np.floor(MinMaxScaler().fit_transform(C_embedded[:,0])*256)

cmin = np.min(C_embedded)
cmax = np.max(C_embedded)
cmin, cmax

(-197.39978, 196.79071)

In [28]:
colors = (C_embedded - cmin) / (cmax - cmin) * 256
colors = np.floor(colors)
colors

array([[145., 153., 212.],
       [131., 145., 256.],
       [242., 192., 131.],
       [ 92.,  16., 159.],
       [231., 102., 183.],
       [204.,  96., 213.],
       [187., 175.,  37.],
       [164., 190., 160.],
       [226., 155., 105.],
       [ 59.,  29., 159.],
       [163.,  32.,  65.],
       [123., 184., 192.],
       [ 90.,  95., 122.],
       [104.,  94.,  17.],
       [167.,  18., 182.],
       [214., 218., 147.],
       [ 86., 146., 113.],
       [121., 149.,  83.],
       [201.,  70., 146.],
       [160., 173., 119.],
       [ 77., 154.,  13.],
       [ 37.,  80., 170.],
       [248., 122., 101.],
       [180.,  70.,  87.],
       [132.,  90., 174.],
       [103., 240., 168.],
       [182., 179., 208.],
       [ 67.,  81., 189.],
       [119., 134., 162.],
       [ 86., 126.,  83.],
       [136., 208., 224.],
       [167., 126., 158.],
       [190., 228.,  79.],
       [181.,  29., 144.],
       [170.,  16., 107.],
       [111., 120., 200.],
       [132.,  19., 131.],
 

In [29]:
X_embedded[0,:]

array([3.3305004, 5.8291836], dtype=float32)

In [30]:
import pandas as pd

In [31]:
columns = ["doi", "title", "year", "link", "authors", "containerTitles", "refCount", "xval", "yval", "rgb", "clusterNo"]
df = pd.DataFrame(index=dois_to_plot, columns=columns)

In [32]:
#from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
#
#max_d = 0.2
#
#Z = linkage(X, method='average', metric='cosine')
##Z = linkage(X, method='complete', metric='cosine')
#clLabels = fcluster(Z, max_d, criterion='distance') - 1
##clLabels = fcluster(Z, 50, criterion='maxclust') - 1
#
#cluster_ids = sorted(np.unique(clLabels))

In [33]:
#from sklearn.cluster import DBSCAN
#
#clLabels = DBSCAN(eps=0.1, min_samples=5, metric="cosine").fit_predict(X)
#cluster_ids = sorted(np.unique(clLabels))

In [34]:
int(np.sqrt(len(dois_to_plot)))

10

In [35]:
from sklearn.mixture import GaussianMixture

n = int(np.sqrt(len(dois_to_plot)))
print(n)

gmm = GaussianMixture(n_components=n, verbose=True)

clLabels = gmm.fit_predict(X)
cluster_ids = sorted(np.unique(clLabels))

10
Initialization 0
Initialization converged: True


In [36]:
[(cl, np.sum(clLabels==cl)) for cl in cluster_ids]

[(0, 12),
 (1, 13),
 (2, 11),
 (3, 5),
 (4, 34),
 (5, 18),
 (6, 8),
 (7, 7),
 (8, 10),
 (9, 2)]

In [37]:
clLabels, np.max(cluster_ids)

(array([1, 6, 0, 8, 3, 3, 3, 5, 3, 1, 4, 1, 5, 7, 1, 6, 0, 5, 4, 5, 9, 7,
        6, 4, 1, 4, 8, 7, 8, 2, 4, 5, 4, 1, 5, 4, 1, 2, 4, 4, 0, 5, 8, 4,
        2, 4, 5, 2, 4, 4, 6, 4, 4, 4, 4, 0, 4, 0, 4, 3, 0, 0, 5, 1, 1, 5,
        1, 2, 8, 8, 1, 5, 0, 9, 1, 2, 5, 4, 0, 6, 5, 8, 7, 7, 7, 5, 2, 8,
        6, 4, 5, 4, 4, 0, 2, 4, 4, 4, 4, 2, 0, 4, 4, 5, 4, 8, 5, 2, 6, 7,
        4, 1, 5, 2, 4, 6, 4, 4, 8, 0], dtype=int64), 9)

In [38]:
from gensim.models.phrases import *
from pprint import pprint
import operator

def phrase_lines(texts_temp):

    phrases = Phrases(texts_temp, min_count=10, threshold=10)
    bigram = Phraser(phrases)
    texts_phrased = [bigram[t] for t in texts_temp]

    phrase_scores = {}
    for phrase, score in phrases.export_phrases(texts_temp):
        phrase_scores[phrase] = score
    print("Bigramme")
    pprint(sorted(phrase_scores.items(), key=operator.itemgetter(1), reverse=True)[:50])

    texts_temp = texts_phrased
    phrases = Phrases(texts_temp, min_count=5, threshold=7)
    trigram = Phraser(phrases)
    texts_phrased = [trigram[t] for t in texts_temp]

    phrase_scores = {}
    for phrase, score in phrases.export_phrases(texts_temp):
        phrase_scores[phrase] = score
    print("Trigramme")
    pprint(sorted(phrase_scores.items(), key=operator.itemgetter(1), reverse=True)[:50])

    return texts_phrased, bigram, trigram

In [39]:
from gensim import corpora, models, similarities

titles = []
for doi in dois_to_plot:
    pub = work_dict[doi]
    if "title" in pub:
        if len(pub["title"]) > 0:
            pub_title = [s for s in pub["title"][0].lower().split() if not s in stopwords]
            titles.append(pub_title)

phrased_lines, bigram, trigram = phrase_lines(titles)

frequency = defaultdict(int)

for line in phrased_lines:
    for term in line:
        frequency[term] += 1

dictionary = corpora.Dictionary(phrased_lines)

corpus = [dictionary.doc2bow(line) for line in phrased_lines]

tfidf = models.TfidfModel(corpus, normalize=True) # step 1 -- initialize a model

2020-03-17 17:03:33,221 : INFO : collecting all words and their counts
2020-03-17 17:03:33,221 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-03-17 17:03:33,227 : INFO : collected 1310 word types from a corpus of 1085 words (unigram + bigrams) and 120 sentences
2020-03-17 17:03:33,227 : INFO : using 1310 counts as vocab in Phrases<0 vocab, min_count=10, threshold=10, max_vocab_size=40000000>
2020-03-17 17:03:33,227 : INFO : source_vocab length 1310
2020-03-17 17:03:33,237 : INFO : Phraser built with 2 phrasegrams
2020-03-17 17:03:33,267 : INFO : collecting all words and their counts
2020-03-17 17:03:33,267 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-03-17 17:03:33,271 : INFO : collected 1317 word types from a corpus of 1035 words (unigram + bigrams) and 120 sentences
2020-03-17 17:03:33,271 : INFO : using 1317 counts as vocab in Phrases<0 vocab, min_count=5, threshold=7, max_vocab_size=40000000>
2020-03-17 17:03:33,271 : INFO :

Bigramme
[(b'novel coronavirus', 12.54399585921325), (b'2019 novel', 11.696428571428571)]
Trigramme
[(b'2019_novel coronavirus', 20.353636363636365),
 (b'novel_coronavirus (2019-ncov)', 13.17),
 (b'wuhan, china', 11.224431818181817)]


In [40]:
clSizes = []
clTitles = defaultdict(list)

for cl in cluster_ids:    
    print("Cluster No.", cl)
    cl_dois = [doi for i, doi in enumerate(dois_to_plot) if clLabels[i] == cl]
    print("Size:", len(cl_dois))
    for doi in cl_dois:
        #pprint(work_dict[doi]["title"])
        pub = work_dict[doi]
        if "title" in pub:
            if len(pub["title"]) > 0:
                pub_title = [s for s in pub["title"][0].lower().split() if not s in stopwords]
                clTitles[cl].append(pub_title)        
    print()
    clSizes.append(len(cl_dois))

Cluster No. 0
Size: 12

Cluster No. 1
Size: 13

Cluster No. 2
Size: 11

Cluster No. 3
Size: 5

Cluster No. 4
Size: 34

Cluster No. 5
Size: 18

Cluster No. 6
Size: 8

Cluster No. 7
Size: 7

Cluster No. 8
Size: 10

Cluster No. 9
Size: 2



In [41]:
clDict = {}

for cl in cluster_ids[:]:
    #cl_titles_phrased = []
    clTokens = []
    if len(clTitles[cl]) < 3:
        continue
    for title in clTitles[cl]:
        title_phrased = trigram[bigram[title]]
        #pprint(title_phrased)
        #cl_titles_phrased.append(title_phrased)
        for t in title_phrased:
            clTokens.append(t)
    #print(clTokens)
    
    cl_dois = [doi for i, doi in enumerate(dois_to_plot) if clLabels[i] == cl]
    
    clBow = dictionary.doc2bow(clTokens)
    clFrequency = {}
    for i, freq in clBow:
        clFrequency[i] = freq
    #print(clBow)
    vec = sorted(tfidf[clBow], key=itemgetter(1), reverse=True)
    
    print("Cluster No.", cl)
    print("Size:", len(cl_dois))
    print("Terms:", ", ".join(["{}".format(dictionary[i]) for i, v in vec[:20] if v >= 100 and clFrequency[i] >= 5]))
    print(", ".join(["{} ({}, {}, {:.2f})".format(dictionary[i], frequency[dictionary[i]], clFrequency[i], v) for i, v in vec[:20]]))
    #print([(dictionary[i], frequency[dictionary[i]], clFrequency[i], v) for i, v in vec])
    
    author_counts = defaultdict(int)
    for doi in cl_dois:
        work = work_dict[doi]
        if "author" in work:
            for author in work["author"]:
                if "family" in author:
                    author_counts[author["family"]] += 1
    top_authors = sorted(author_counts.items(), key=itemgetter(1), reverse = True)
    top_authors = [(a, n) for a,n in top_authors if n >= 2][:10]
    print("top_authors:", top_authors)
    
    print()
    
    
    clDict[cl] = {}
    clDict[cl]["id"] = cl
    clDict[cl]["size"] = len(cl_dois)
    clDict[cl]["dois"] = cl_dois
    clDict[cl]["vec"] =  [(dictionary[i], frequency[dictionary[i]], clFrequency[i], v) for i, v in vec]
    clDict[cl]["top_authors"] = top_authors
    

Cluster No. 0
Size: 12
Terms: 
fusion (2, 2, 0.24), genome (3, 2, 0.18), associated (4, 2, 0.17), respiratory (4, 2, 0.17), new (5, 2, 0.16), origin (5, 2, 0.16), emerging (7, 2, 0.14), coronaviruses (8, 2, 0.14), sars (8, 2, 0.14), coronavirus (22, 3, 0.13), compared (1, 1, 0.12), higher (1, 1, 0.12), reproductive (1, 1, 0.12), learning (1, 1, 0.12), past: (1, 1, 0.12), possible (1, 1, 0.12), severe (1, 1, 0.12), urgent (1, 1, 0.12), assessment (1, 1, 0.12), b (1, 1, 0.12)
top_authors: [('Liu', 8), ('Wang', 7), ('Chen', 6), ('Zhang', 5), ('Wu', 4), ('Xu', 3), ('Li', 3), ('Zhu', 3), ('Jiang', 3), ('Shi', 3)]

Cluster No. 1
Size: 13
Terms: 
2019‐ncov (5, 3, 0.23), coronavirus (22, 5, 0.21), transmission (7, 3, 0.21), (2019‐ncov) (2, 2, 0.20), global (4, 2, 0.17), evolution (4, 2, 0.17), health (6, 2, 0.16), outbreak (16, 3, 0.15), novel (6, 2, 0.15), emerging (7, 2, 0.14), wuhan,_china (8, 2, 0.13), coronaviruses: (1, 1, 0.12), pathogenesis (1, 1, 0.12), replication, (1, 1, 0.12), struc

In [42]:
from bokeh.colors import RGB

#year2alpha = lambda year: 1.5 / (1. + np.exp(0.2*(2018 - year))) + 0.25
year2alpha = lambda year: np.clip(2. / (1. + np.exp(0.10*(2018 - year))), a_min=0.1, a_max=None)


for i, doi in enumerate(dois_to_plot):
    pub = work_dict[doi]
    df.at[doi, "doi"] = doi
    df.at[doi, "title"] = pub["title"][0]
    if "container-title" in pub:
        df.at[doi, "containerTitle"] = ";".join(pub["container-title"])
    #line = pub["title"][0]
    #n = 8
    #df.at[doi, "title"] = "\n".join([line[j:j+n] for j in range(0, len(line), n)])
    df.at[doi, "issued"] = "-".join([str(x) for x in pub["issued"]["date-parts"][0]])
    df.at[doi, "refCount"] = ref_counts[doi]
    df.at[doi, "refCountNormalized"] = ref_count_normalized[doi]
    #df.at[doi, "scatterSize"] = 6.+ np.power(ref_counts[doi], 0.8)
    df.at[doi, "scatterSize"] = 4+2.5*np.power(ref_count_normalized[doi], 1.)
    df.at[doi, "xval"] = X_embedded[i,0]
    df.at[doi, "yval"] = X_embedded[i,1]
    df.at[doi, "rgb"] = RGB(colors[i,0], colors[i,1], colors[i,2])
    df.at[doi, "alpha"] = year2alpha(pub["issued"]["date-parts"][0][0])
    try:
        pub_authors = [a["family"] for a in pub["author"]]
        df.at[doi, "author"] = ", ".join(pub_authors)
    except KeyError:
        df.at[doi, "author"] = "unavailable"
    
    df.at[doi, "clusterNo"] = clLabels[i]
    

[year2alpha(y) for y in (2018, 2017, 2014, 2010, 2000, 1980)]

[1.0,
 0.95004162504212,
 0.802624679775096,
 0.6200510377447751,
 0.28370212980097553,
 0.1]

In [43]:
#df["scatterSize"] *= 0.75
df["scatterSize"] *= 1.5

In [44]:
df["xval"].max()
df["xval"].min()
x_span = df["xval"].max() - df["xval"].min()
y_span = df["yval"].max() - df["yval"].min()

In [45]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [46]:
#tsne.transform

In [47]:
##### clDict = pickle.load(open("clDict.p", "rb"))

label_source_df = pd.DataFrame(index=clDict.keys(), columns=["x", "y", "text"])

for cl_id, cluster in clDict.items():
    #print(cluster)
    cl_dois = cluster["dois"]
    x = np.zeros(len(cl_dois))
    y = np.zeros(len(cl_dois))
    w = np.zeros(len(cl_dois))
    r = np.zeros(len(cl_dois))
    g = np.zeros(len(cl_dois))
    b = np.zeros(len(cl_dois))
    n = 0
    
    
    for i, doi in enumerate(cl_dois):
        if doi in df.index:
            n += 1
            x[i] = df.at[doi, "xval"]
            y[i] = df.at[doi, "yval"]
            w[i] = df.at[doi, "refCountNormalized"]
        else:
            w[i] = 0
    #if n < 3:
    #    continue
    
    xx = np.median(x)
    yy = np.median(y)
    
    #xx = np.average(x, weights=w)
    #yy = np.average(y, weights=w)
    
    x_std = np.std(x/x_span)
    y_std = np.std(y/y_span)
    
    print()
    print(cl_id)
    print(", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.1 and totalFreq>=3][:5]))
    print(xx,yy,len(cl_dois))
    print(x_std, y_std , np.sqrt(x_std**2 + y_std**2))
    print(np.sum(w))
    
    
    if np.sum(w) < 10:
        print("w too small")
        continue
    
    std_max = 0.1
    
    if x_std**2 + y_std**2 > std_max**2:
        print("std too big")
        #continue
    
    
    
    label_source_df.at[cl_id, "x"] = xx
    label_source_df.at[cl_id, "y"] = yy
    #label_source.at[cl_id, "text"] = ", ".join([s for s, n in cluster["top_authors"][:3]]) \
    #    + " | " + ",\n".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.25 and clFreq>=3][:2])
    
    #label_source_df.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.3])
    label_source_df.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.1 and totalFreq>=3][:2])
    #label_source.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.225 and clFreq>=2][:2])
    #label_source.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.01 and clFreq>=1][:1])
    #label_source.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"]][:1])
    #label_source.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=500])
    if len(label_source_df.at[cl_id, "text"]) == 0:
        label_source_df.at[cl_id, "text"] = np.nan
    
    top_authors = cluster["top_authors"]
    label_source_df.at[cl_id, "authors"] = ", ".join([a for a,n in top_authors][:3])
    
    #print(xx,yy,len(cl_dois),label_source_df.at[cl_id, "text"])
    

                 
    #xy = X_embedded[, dtype=int), :]
    #x = df["xval"].loc[cl_dois]
    #print()

#label_source = 
label_source_df = label_source_df[pd.notnull(label_source_df["x"])]
label_source_df = label_source_df[pd.notnull(label_source_df["text"])]

    


0
genome, associated, respiratory, new, origin
-2.745163083076477 5.540922403335571 12
0.18661165856519799 0.16497473041201782 0.24907945075210838
nan
std too big

1
2019‐ncov, coronavirus, transmission, global, evolution
3.382181167602539 5.644922733306885 13
0.10140738342497796 0.0648985402595606 0.12039633690823946
45.20961281708946
std too big

2
2020, 2019_novel_coronavirus, cluster, epidemiology, report
-4.566805362701416 -5.64125394821167 11
0.043308210425842764 0.029827800981839937 0.05258610844796682
15.714953271028035

3
bat, coronaviruses, sars-like, origin, coronavirus
-0.5437428951263428 10.287522315979004 5
0.03726471277301414 0.029495540896931963 0.04752521173922142
nan

4
infection, china, coronavirus:, response, patients
0.2362801656126976 0.2737620174884796 34
0.1772500546280697 0.1359565069513308 0.22338700420584218
35.56542056074766
std too big

5
clinical, pneumonia, (2019-ncov), infected, patients
-2.148265242576599 1.0412233471870422 18
0.1264463265593748 0.1069

In [48]:
from bokeh.plotting import figure, output_file, show, save, output_notebook
from bokeh.models import HoverTool, Range1d, LabelSet, LassoSelectTool, CustomJS, TapTool, OpenURL
from bokeh.models.sources import ColumnDataSource
from bokeh.io import reset_output

reset_output()


#class hover(HoverTool):
#    hover.tooltips = [
#    ("index", "$index"),


hover = HoverTool()
hover.tooltips = [
    ("Title", "@title"),
    ("Autoren", "@author"),
    ("Veröffentlicht", "@issued"),
    ("Journal", "@containerTitle"),
    ("Referenzen", "@refCount"),
]




s1 = ColumnDataSource(data=df)


callback = CustomJS(code="""
// the event that triggered the callback is cb_obj:
// The event type determines the relevant attributes
console.log('Tap event occured at x-position: ' + cb_obj.x)
""")


def lasso_cb(x):
    print(x)

#lasso = LassoSelectTool(callback=lasso_cb)
lasso = LassoSelectTool()


p = figure(plot_width=1200, plot_height=1000, active_scroll = "wheel_zoom")
#p = figure(plot_width=1200, plot_height=1000, output_backend="webgl", active_scroll = "wheel_zoom")
p.circle(x='xval', y='yval', size="scatterSize", color="rgb", alpha="alpha", source=s1)

p.add_tools(hover)
p.add_tools(lasso)

p.js_on_event('tap', callback)

#p.toolbar.active_scroll = "wheel_zoom"

df["xval"].max()

left, right, bottom, top = -120, 120, -90, 90
left, right, bottom, top = -80, 80, -60, 60
left, right, bottom, top = -100, 100, -90, 90
p.x_range=Range1d(left, right)
p.y_range=Range1d(bottom, top)

try:
    label_source = ColumnDataSource(label_source_df)   

    p.scatter(x='x', y='y', source=label_source, size=8, color="black", marker="diamond")
    
    labels = LabelSet(x='x', y='y', x_offset=-20, y_offset=5, text='text', level='glyph', 
                  source=label_source, render_mode='canvas')
    #author_labels = LabelSet(x='x', y='y', x_offset=10, y_offset=-15, text='authors', level='glyph', 
    #              source=label_source, render_mode='canvas')

    p.add_layout(labels)
    #p.add_layout(author_labels)
except ValueError:
    pass
except NameError:
    pass

output_file("test.html")
#output_notebook()
#show(p)
#save(p)

from bokeh.layouts import widgetbox, column, row
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn

s2 = ColumnDataSource(df)

columns = [
        TableColumn(field="clusterNo", title="Cluster"),
        TableColumn(field="title", title="Title"),
        TableColumn(field="author", title="Autoren"),
        TableColumn(field="issued", title="Veröffentlichungsjahr"),
    ]
data_table = DataTable(source=s1, columns=columns, width=640, height=800)

#p.add_widget(widgetbox(data_table))

#show(p)


#s1.callback = 
jscode = CustomJS(args=dict(s2=s2), code="""
    var inds = cb_obj.get('selected')['1d'].indices;
    var d1 = cb_obj.get('data');
    var d2 = s2.get('data');
    
    
    d2["title] = []
    d2["author] = []
    d2["issued] = []
    for (i = 0; i < inds.length; i++) {
        d2["title"].push(d1["title"][inds[i]])
        d2["author"].push(d1["author"][inds[i]])
        d2["issued"].push(d1["issued"][inds[i]])
    }
    s2.trigger('change');
    
""")

p.add_tools(TapTool(callback=OpenURL(url="http://doi.org/@doi")))

# use the "color" column of the CDS to complete the URL
# e.g. if the glyph at index 10 is selected, then @color
# will be replaced with source.data['color'][10]
#url = "http://doi.org/@doi"
#taptool = p.select(type=TapTool)
##taptool = TapTool()
#taptool.callback = OpenURL(url=url)
##p.add_tools(taptool)

layout = row(p, widgetbox(data_table))
#layout = column(p, widgetbox(data_table))

show(layout)
#save(layout)

2020-03-17 17:03:33,906 : INFO : Session output file 'test.html' already exists, will be overwritten.


In [49]:
break

SyntaxError: 'break' outside loop (<ipython-input-49-6aaf1f276005>, line 4)

In [ ]:
df.sample(1)

In [ ]:
vars(s1)
#s1.data

In [ ]:

label_source
#type(label_source)

In [ ]:
columns = [
        TableColumn(field="title", title="Title"),
        TableColumn(field="author", title="Autoren"),
        TableColumn(field="issued", title="Jahr", formatter=DateFormatter()),
    ]
data_table = DataTable(source=ColumnDataSource(df), columns=columns, width=400, height=280)

show(widgetbox(data_table))

In [ ]:
#df_all = pd.DataFrame(index=set(k for k in work_dict.keys() if k in df), columns=["doi", "clusterID", "issued", "title", "authors", "containerTitles", "refCount", "link", "inPlot"])
#for i, doi in enumerate(work_dict.keys()):

#df_dois = set(dois_to_plot & work_dict.keys())
df_dois = set(reference_vectors.vocab.keys() & work_dict.keys())

df_all = pd.DataFrame(index=df_dois, columns=["doi", "issued", "issuedYear", "title", "authors", "containerTitles", "refCount", "refCountNormalized", "link", "clusterID", "clusterKWs", "inPlot"])
for i, doi in enumerate(df_dois):
    
    if i % 1000 == 0:
        print(i, len(df_dois))
    
    pub = work_dict[doi]
    #print(pub)
    df_all.at[doi, "doi"] = doi
    
    
    
    
    try:
        df_all.at[doi, "title"] = pub["title"][0]
    except IndexError:
        pass
        #df_all.at[doi, "title"] = None
    except KeyError:
        pass
    df_all.at[doi, "issued"] = "-".join([str(x) for x in pub["issued"]["date-parts"][0]])
    df_all.at[doi, "issuedYear"] = pub["issued"]["date-parts"][0][0]
    df_all.at[doi, "refCount"] = ref_counts[doi]
    try:
        pub_authors = [a["family"] for a in pub["author"]]
        df_all.at[doi, "authors"] = ", ".join(pub_authors)
    except KeyError:
        df_all.at[doi, "authors"] = "unavailable"
    try:
        df_all.at[doi, "containerTitles"] = pub["container-title"]
    except KeyError:
        pass
    try:
        df_all.at[doi, "link"] = "=Hyperlink(\"https://doi.org/{}\")".format(doi)
    except KeyError:
        pass
    
    if doi in dois_to_plot:
        df_all.at[doi, "inPlot"] = True
        clid = df.at[doi, "clusterNo"]
        df_all.at[doi, "clusterID"] = clid
        if clid in clDict:
            df_all.at[doi, "clusterKWs"] = ", ".join([s for s, totalFreq, clFreq, v in clDict[clid]["vec"] if v>=0.1 and totalFreq>=3])
    else:
        df_all.at[doi, "inPlot"] = False
        
    try:
        year = pub["issued"]["date-parts"][0][0]
        x = ref_counts[doi]
        if year in lambda_mle_ref_count:
            l = lambda_mle_ref_count[year]
        else:
            l = lambda_mle_ref_count_default
        #df_all.at[doi, "expRefCountPercentile"] = 1. - np.exp(-l*x)
        df_all.at[doi, "refCountNormalized"] = l*x
    except KeyError:
        pass

In [ ]:
df_all = df_all.sort_values(by="refCount", ascending=False)
df_all.sample(10)
df_all.to_excel("df_all.xlsx")

In [ ]:
df_all_inPlot = df_all[df_all["inPlot"]==True]
len(df_all_inPlot)

In [ ]:
df_all_inPlot.to_excel("df_all_inPlot.xlsx")

In [ ]:
df_all.sample(10)

In [ ]:
dois_to_plot.index("10.1021/op200097d")

Idee:

für Bestimmung der Wahrscheinlichkeit für Referenz eines Papers bei einer vorhandenen Menge an bekannten Referenzen

Kreuzvalidierung möglicher Metriken mit Subset der Referenz-Relationen